![](header.jpg)

# Modified Davient-Hartenberg Equations

Kevin Walchko, Phd

10 July 2017

---

Denavit Hartenberg (DH) is an attempt to standardize how we represent serial manipulators (i.e., robot arms). It is typically one of the first ways you learn. It is really easy (methodical) to do forward kinematics, but becomes more challenging when doing inverse kinematics. Here we are going to introduce what is goning on, but you need to focus on the DH process. If you follow the process, then all will work out fine. Don't get too hung up on the begining math, understand the concepts so you can follow the DH process.

## References

- Wikipedia: [Modified DH](https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters#Modified_DH_parameters)
- [darpa robot challenge](https://www.youtube.com/watch?v=diaZFIUBMBQ)
- [darpa robot fails](https://youtu.be/wX0KagJ1du8)
- [Walking robot](https://vimeo.com/194676675)


$$
T^A_B = \begin{bmatrix}
   & R^A_B & & P^A_B \\
  0 & 0 & 0 & 1
\end{bmatrix} \\
\begin{bmatrix}
  P^A \\
  1
\end{bmatrix} = 
T^A_B
\begin{bmatrix}
  P^B \\
  1
\end{bmatrix}
$$

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/DHParameter.png/609px-DHParameter.png)

## Process (Craig, section 3.4 & 3.6, pg 67)

Now the process laid out in Craig's book, defines some parameters for eash link in a robot arm:

| | | |
|------------|:--------------|:------------------------------------------------------|
| $a_i$      | link length   | distance from $z_i$ to $z_{i+1}$ measured along $x_i$ |
| $d_i$      | offset        | distance from $x_{i-1}$ to $x_i$ along $z_i$          |
| $\alpha_i$ | twist         | angle from $z_i$ to $z_{i+1}$ measured about $x_i$    |
| $\theta_i$ | rotation      | angle from $x_{i−1}$ to $x_i$ measured about $z_i$    |

Summary of steps:

1. Identify the joint axes and imagine (or draw) infinite lines along them. For steps 2 through 5 below, consider two of these neighboring lines (at axes i and i+1).
1. Identify the comon perpendicular betwen them,or point of intersection. At the point of intersection, or at the point where the comon perpendicular meets the $i^{th}$ axis, asign the link-frame origin.
1. Asign the $Z_i$ axis pointing along the $i^{th}$ joint axis.
1. A sign the axis pointing along the comon perpendicular, or, if the axes intersect, asign k1 to be normal to the plane containing the two axes.
1. Asign the y axis to complete a right-hand cordinate system ... honestly don't draw y-axis 
1. Asign {0} to match {1} when the first joint variable is zero. For {N}, chose an origin location and $X_N$ direction freely, but generaly so as to cause as many linkage parameters as possible to become zero.

Now, once you have the parameters, you can enter them into the followng matrix to get the relationship between frame i and frame i+1. Note, that these are **not** euler angles, but rather:

1. A translation along z by d
1. Rotation about z by $\theta$
1. translation along x by a
1. Rotation about x by $\alpha$

This sequence turns into the the following homogenious transform:

$$
\begin{eqnarray}
T^{i-1}_i = R_x(\alpha_{i-1}) Translate_x(a_{i-1}) R_z(\theta_i) Translate_z(d_i) \\
\\
T^{i-1}_i = \begin{bmatrix}
  \cos(\theta_i)                   & -\sin(\theta_i)                  & 0                   & a_{i-1} \\
  \sin(\theta_i)\cos(\alpha_{i-1}) & \cos(\theta_i)\cos(\alpha_{i-1}) & -\sin(\alpha_{i-1}) & -\sin(\alpha_{i-1})d_i \\
  \sin(\theta_i)\sin(\alpha_{i-1}) & \cos(\theta_i)\sin(\alpha_{i-1}) & \cos(\alpha_{i-1})  & \cos(\alpha_{i-1})d_i \\
  0                                & 0                                & 0                   & 1
\end{bmatrix}
\end{eqnarray}
$$

You will create 1 matrix for each link in your serial manipulator. Then you can multiply these matricies together to get the transform from base frame {0} to end effector frame {3} by: $T^0_3 = T^0_1 T^1_2 T^2_3$

Also, the general format of **every** homogenious matrix is:

$$
T_{4x4} =
\begin{bmatrix}
  R_{3x3} & t_{3x1} \\
  0_{1x3} & 1
\end{bmatrix}
$$

where $R$ is your rotation matrix, $t$ is your translation, and T is your homogenious matrix.

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
from numpy.testing import assert_allclose
from math import pi, sin, cos, atan2, asin, acos
from matplotlib import pyplot as plt

## Simple 2D Example

Following the process for assigning frames to a manipulator, you get the following:

![](dh_pics/2d-ex.png)

Looking at the above frames, they are related by:

| Link | $a_{i-1}$ | $\alpha_{i-1}$ | $d_i$ | $\theta_i$ |
|------|-----------|----------------|-------|------------|
| 1    | 0     | 0          | 0     | $\theta_1$ |
| 2    | $a_1$ | 0          | 0     | $\theta_2$ |
| 3    | $a_2$ | 0          | 0     | 0          |

Now using Craig eqn 3.6, we can substitute these values in and get the relationship between the inertial frame and the end effector. However note, $\theta_i$ are variable parameters. Typically we would simplify these equations down leaving only the $\theta_i$ parameters. Let's use the python symbolic toolbox to generate the equations of motion.

In [8]:
# Let's grab some libraries to help us manipulate symbolic equations
import numpy as np
import sympy
from sympy import symbols, sin, cos, pi, simplify

def makeT(a, alpha, d, theta):
    # create a modified DH homogenious matrix
    # this is the same matrix as above
    return np.array([
        [           cos(theta),           -sin(theta),           0,             a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
        [sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  d*cos(alpha)],
        [                    0,                     0,           0,             1]
    ])

def simplifyT(tt):
    """
    This goes through each element of a matrix and tries to simplify it.
    """
    ret = tt.copy()
    for i, row in enumerate(tt):
        for j, col in enumerate(row):
            ret[i,j] = simplify(col)
    return ret

def subs(tt, m):
    """
    This allows you to simplify the trigonomic mess that kinematics can
    create and also substitute in some inputs in the process
    
    Yes, this is basically the same as above. I could combine these into 1
    function, but I wanted to beclearer on what I am doing.
    """
    ret = tt.copy()
    for i, row in enumerate(tt):
        for j, col in enumerate(row):
            try:
                ret[i,j] = col.subs(m)
            except:
                ret[i,j] = simplify(col)
    return ret

In [9]:
# make thetas (t) and link lengths (a) symbolics
t1, t2 = symbols('t1 t2')
a1, a2 = symbols('a1 a2')

# let's create our matrices
T1 = makeT(0, 0, 0, t1)
T2 = makeT(a1, 0, 0, t2)
T3 = makeT(a2, 0, 0, 0)

# T13 = T1 * T2 * T3
T13 = T1.dot(T2.dot(T3))

print('T1 = ', T1)
print('T2 = ', T2)
print('T3 = ', T3)
print('\nSo the combined homogenious matrix is:\n')
print('T13 = ', T13)

T1 =  [[cos(t1) -sin(t1) 0 0]
 [sin(t1) cos(t1) 0 0]
 [0 0 1 0]
 [0 0 0 1]]
T2 =  [[cos(t2) -sin(t2) 0 a1]
 [sin(t2) cos(t2) 0 0]
 [0 0 1 0]
 [0 0 0 1]]
T3 =  [[1 0 0 a2]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]

So the combined homogenious matrix is:

T13 =  [[-sin(t1)*sin(t2) + cos(t1)*cos(t2) -sin(t1)*cos(t2) - sin(t2)*cos(t1) 0
  -a2*sin(t1)*sin(t2) + (a1 + a2*cos(t2))*cos(t1)]
 [sin(t1)*cos(t2) + sin(t2)*cos(t1) -sin(t1)*sin(t2) + cos(t1)*cos(t2) 0
  a2*sin(t2)*cos(t1) + (a1 + a2*cos(t2))*sin(t1)]
 [0 0 1 0]
 [0 0 0 1]]


In [11]:
ans = simplifyT(T13)
print(ans)
print('-'*25)
print('position x: {}'.format(ans[0,3]))
print('position y: {}'.format(ans[1,3]))

[[cos(t1 + t2) -sin(t1 + t2) 0 a1*cos(t1) + a2*cos(t1 + t2)]
 [sin(t1 + t2) cos(t1 + t2) 0 a1*sin(t1) + a2*sin(t1 + t2)]
 [0 0 1 0]
 [0 0 0 1]]
-------------------------
position x: a1*cos(t1) + a2*cos(t1 + t2)
position y: a1*sin(t1) + a2*sin(t1 + t2)


In [12]:
# what if I wanted to substitute in an angle?
# just give it an array of tuples
ans = subs(T13, [(t1, 0)])  # here it is only t1, but I could do: [(t1, angle), (t2, angle)]
print(ans)
print('-'*25)
print('position x: {}'.format(ans[0,3]))
print('position y: {}'.format(ans[1,3]))

[[cos(t2) -sin(t2) 0 a1 + a2*cos(t2)]
 [sin(t2) cos(t2) 0 a2*sin(t2)]
 [0 0 1 0]
 [0 0 0 1]]
-------------------------
position x: a1 + a2*cos(t2)
position y: a2*sin(t2)


# KUKA KR270 Example

![](dh_pics/kuka-kr270.png)

Now following the steps above, we get the complete DH table below. Notice, the distances in the picture (d) have been changed to m, because I didn't want people getting confused between distance d and DH parameter d. Therefore everything is now distance m to remove confusion:

| $Link$ | $a_{i-1}$ | $\alpha_{i-1}$  | $d_i$ | $\theta_i$ |
|:------|:----------|:---------------|:------|:-----------|
| 1    | 0         | 0              | $m_1$ | $\theta_1$ |
| 2    | $m_2$     | 90             | 0     | $\theta_2$ |
| 3    | $m_3$     | 0              | 0     | $\theta_3$ |
| 4    | $m_5$     | -90            | $m_4$ | $\theta_4$ |
| 5    | 0         | 90             | 0     | $\theta_5$ |
| 6    | 0         | -90            | $m_6$ | $\theta_6$ |

A step-by-step walk through is found [here](lsn7-extra.pdf) (this is included in the zip file and called `lsn7-extra.pdf`), make sure you understand this process because you will be tested on it. Also, I strongly suggest you do not wait until the night before the test to learn this!